<a href="https://colab.research.google.com/github/jquilapan/covid/blob/master/Covid.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **1.- Paquetes a utilizar**
Se importan los paquetes que se usarán.

In [67]:
import pandas as pd
import numpy as np
from sklearn.svm import SVR

# **2.- Dataset**
Se declara la url de donde proviene el archivo a importar, en este caso, se extrae del github del Ministerio de Ciencias de Chile.

In [68]:
url = 'https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto5/TotalesNacionales_T.csv'

In [69]:
data = pd.read_csv(url)
data = data[['Fecha', 'Casos nuevos totales']]
data = data.reset_index(drop=True)
data['correlativo'] = np.arange(len(data))
data['Fecha'] = pd.to_datetime(data['Fecha'])
data = data.rename(columns={'Casos nuevos totales':'casos_totales'})

# **3.- Modelo a Utilizar**
Se declara el modelo a utilizar, para este ejemplo, usaremos el modelo Support Vector Regressor.

In [70]:
svr_lin = SVR(C=1e3)

In [71]:
correlativo = data['correlativo'].tolist()
casos = data['casos_totales'].tolist()
correlativo_2 = np.reshape(correlativo, (len(correlativo), 1))
svr_lin.fit(correlativo_2, casos)
for x in range(7):
  predict_value = [max(correlativo) + 1]
  predict_value = np.reshape(predict_value, (len(predict_value), 1))
  pred = svr_lin.predict(predict_value)[0]
  correlativo.append(max(correlativo)+1)
  casos.append(pred)
forecast = pd.DataFrame(list(zip(correlativo, casos)), columns=['correlativo', 'svr'])

In [72]:
last_month = max(data['Fecha'].unique())
last_month = pd.to_datetime(last_month)

In [73]:
last_correlativo = max(data.correlativo.unique())

In [74]:
meses_predecir = []
numeros_predecir = []
for dia in range(0, 7):
  meses_predecir.append(last_month + pd.tseries.offsets.DateOffset(days=dia+1))
  numeros_predecir.append(last_correlativo + dia+1)

In [75]:
numeros_meses = pd.DataFrame(list(zip(meses_predecir, numeros_predecir)), columns=['Fecha','correlativo'])

In [76]:
forecast_final = pd.merge(numeros_meses, forecast, left_on='correlativo', right_on='correlativo', how='left')

In [77]:
forecast_final

,Fecha,correlativo,svr
0,2021-06-13,468,6781.072550
1,2021-06-14,469,6777.363029
2,2021-06-15,470,6773.195845
3,2021-06-16,471,6768.574726
4,2021-06-17,472,6763.503505
5,2021-06-18,473,6757.986126
6,2021-06-19,474,6752.026636
